# 事前準備

## ライブラリ

In [1]:
!pip install --upgrade xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 12.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


## EXCELを処理する関数

国土交通省ホームページのEXCELデータから作業する

輸出・輸入・移出・移入とも同じ構成のExcelになっているので、EXCELのインポートからDataFameの整形までは共通関数でOK

In [26]:
import pandas as pd

def get_excel(url):
  # ブックの指定
  input_file_name = url
  input_book = pd.ExcelFile(input_file_name)
  
  # 統計結果は2番目のシート、上から2行は無視
  input_sheet_name = input_book.sheet_names
  df = input_book.parse(input_sheet_name[1],skiprows = 2)

  df = df.astype({'トン数': 'int'})

  return df

## CSVに保存する関数

In [42]:
from google.colab import files

def download_csv(df, fileName):
  df.to_csv(fileName, encoding="shift_jis")
  files.download(fileName)
  return

# 単年度分析

## 輸入

### 港別✕品種別

In [37]:
import pandas as pd
import numpy as np

def import_kind(time, url, prefecture=None):

  # EXCELをインポート
  df = get_excel(url)

  # 集計
  df = pd.pivot_table(df, index=['都道府県', '港名'], columns='品種名', values='トン数', margins=True, aggfunc=np.sum).reset_index()

  # 都道府県指定がある場合
  if prefecture is not None:
    df = df[df['都道府県'] == prefecture]
  
  # 欠損列を削除
  df = df.dropna(how='all', axis=1)

  return df

In [43]:
# 年次・URLを指定
time = '2022年度'
url = 'https://www.mlit.go.jp/k-toukei/content/001448038.xlsx'

# 全港湾の集計結果を取得
# df = import_kind(time,url)

# 兵庫県の集計結果を取得
df = import_kind(time,url,'兵庫県')

# csvに保存
fileName = f'港別✕品種別の輸入量【{time}】.csv'
download_csv(df, fileName)

df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

品種名,都道府県,港名,がん具,その他の石油,その他日用品,その他林産品,その他機械,その他畜産品,その他石油製品,その他繊維工業品,...,鋼材,陶磁器,電気機械,非金属鉱物,非鉄金属,飲料,麦,ＬＮＧ（液化天然ガス）,ＬＰＧ（液化石油ガス）,All
7,兵庫県,姫路,NaN,NaN,NaN,NaN,NaN,NaN,12031.0,NaN,...,4270.0,NaN,96.0,33976.0,NaN,NaN,NaN,16935069.0,3626.0,19253280
8,兵庫県,尼崎西宮芦屋,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29786
9,兵庫県,東播磨,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17021.0,NaN,NaN,528134.0,NaN,NaN,NaN,2019.0,37560.0,15515389
10,兵庫県,相生,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12123
11,兵庫県,神戸,163990.0,9787.0,198550.0,5196.0,7856.0,785144.0,36648.0,443891.0,...,56214.0,44405.0,1008754.0,149818.0,304056.0,278761.0,1300287.0,NaN,278741.0,26041274
12,兵庫県,赤穂,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41360.0,NaN,NaN,NaN,NaN,NaN,416200


### 港別✕仕向国別（品種指定）

In [18]:
import pandas as pd
import numpy as np

def import_country(url,kind):

  # EXCELをインポート
  df = get_excel(url)

  # 品種名（kind）に一致する行を抽出
  df = df[df['品種名'] == kind]

  # 集計
  df = pd.pivot_table(df, index=['都道府県', '港名'], columns='仕向国', values='トン数', margins=True, aggfunc=np.sum).reset_index()

  # csvに保存
# from google.colab import files
# df.to_csv('兵庫県の総人口.csv', encoding="shift_jis")
# files.download('兵庫県の総人口.csv')

  return df


In [19]:
# URLを指定
url = 'https://www.mlit.go.jp/k-toukei/content/001448038.xlsx'

# 品種名を指定
kind = 'ＬＮＧ（液化天然ガス）'

df = import_kind(url,kind)
df

仕向国,都道府県,港名,アメリカ,アラブ首長国,イギリス,インドネシア,エジプト,オマーン,オーストラリア,カタール,...,パプアニューギニア,フィリピン,フランス,ブルネイ,ペルー,マレーシア,ロシア,台湾,諸国,韓国
0,三重県,四日市,959059.0,NaN,NaN,674544.0,NaN,449991.0,3878849.0,1578185.0,...,501073.0,165268.0,145220.0,NaN,172072.0,2525663.0,777126.0,NaN,NaN,167635.0
1,兵庫県,姫路,1145498.0,NaN,NaN,2146386.0,152857.0,327576.0,7492266.0,1376415.0,...,1386312.0,NaN,NaN,811560.0,176614.0,1287183.0,632402.0,NaN,NaN,NaN
2,兵庫県,東播磨,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
3,北海道,石狩湾新,167583.0,NaN,NaN,NaN,NaN,NaN,907248.0,NaN,...,NaN,NaN,NaN,241811.0,NaN,276314.0,1241826.0,NaN,NaN,NaN
4,千葉県,千葉,NaN,NaN,NaN,NaN,NaN,299726.0,7091432.0,2739978.0,...,NaN,NaN,NaN,6593875.0,NaN,1098779.0,930008.0,NaN,NaN,NaN
5,千葉県,木更津,1645425.0,737300.0,NaN,NaN,NaN,139764.0,7119700.0,5218668.0,...,1915201.0,NaN,NaN,154776.0,NaN,1930926.0,933136.0,NaN,NaN,NaN
6,大分県,大分,69634.0,NaN,NaN,229882.0,NaN,NaN,1103274.0,NaN,...,NaN,NaN,NaN,NaN,NaN,65494.0,257240.0,NaN,NaN,NaN
7,大阪府,堺泉北,477295.0,NaN,NaN,186117.0,NaN,1022465.0,3306830.0,274748.0,...,616110.0,NaN,NaN,246176.0,NaN,690651.0,384241.0,NaN,NaN,NaN
8,宮城県,仙台塩釜,72051.0,NaN,NaN,NaN,NaN,NaN,66354.0,308193.0,...,NaN,NaN,NaN,NaN,NaN,423145.0,139483.0,NaN,NaN,NaN
9,富山県,伏木富山,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,567200.0,NaN,NaN,NaN,NaN
